In [1]:
"""!pip install aeon
!pip install sktime
!pip install tsfresh
!pip install tslearn
!pip install PyWavelets"""

'!pip install aeon\n!pip install sktime\n!pip install tsfresh\n!pip install tslearn\n!pip install PyWavelets'

### To Do list


*   Comparar os resultados do 1NN contra o SVM+RF
*   Comparar os resultados dos classificadores Feature Based com o SVM+RF
*   Comparar os resultados do MetaClf_Conc contra o MetaClf_Dict



In [2]:
import pandas as pd
import numpy as np

from aeon.datasets import load_classification
from aeon.datasets.tsc_data_lists import univariate_equal_length
from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier
from aeon.classification.convolution_based import RocketClassifier

from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import MinimalFCParameters

from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import PiecewiseAggregateApproximation, SymbolicAggregateApproximation

import os
import math
import pywt

from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV

from scipy.fftpack import fft
from scipy.stats import norm


from tqdm import tqdm
import timeit
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")

### Function Transform_data Original

In [3]:
# Transform data original
"""def sax_transform(series, w, a):
    paa = [series[i:i + w].mean() for i in range(0, len(series), w)]

    if np.std(paa) != 0:
        paa = (paa - np.mean(paa)) / np.std(paa)
    else:
        paa = paa - np.mean(paa)

    breakpoints = norm.ppf(np.linspace(0, 1, a+1)[1:-1])
    sax_symbols = np.array(range(a))
    sax_representation = sax_symbols[np.digitize(paa, breakpoints)]

    return sax_representation

def transform_data(X, num_features=10):
    a = 5
    w = int(X.shape[1] / num_features)  # Ajuste do tamanho da janela baseado no número de características desejado

    X_sax = np.array([sax_transform(row, w, a) for row in X])
    X_fft = np.abs(fft(X, axis=1))

    coeffs_cA, coeffs_cD = pywt.dwt(X, 'db1', axis=1)
    X_dwt = np.hstack((coeffs_cA, coeffs_cD))

    X_paa = np.column_stack([X[:, i:i+2].mean(axis=1) for i in range(0, X.shape[1], 2)])

    return {
        "TS": X,
        "FFT": X_fft,
        "DWT": X_dwt,
        "PAA": X_paa,
        "SAX": X_sax
    }"""

'def sax_transform(series, w, a):\n    paa = [series[i:i + w].mean() for i in range(0, len(series), w)]\n\n    if np.std(paa) != 0:\n        paa = (paa - np.mean(paa)) / np.std(paa)\n    else:\n        paa = paa - np.mean(paa)\n\n    breakpoints = norm.ppf(np.linspace(0, 1, a+1)[1:-1])\n    sax_symbols = np.array(range(a))\n    sax_representation = sax_symbols[np.digitize(paa, breakpoints)]\n\n    return sax_representation\n\ndef transform_data(X, num_features=10):\n    a = 5\n    w = int(X.shape[1] / num_features)  # Ajuste do tamanho da janela baseado no número de características desejado\n\n    X_sax = np.array([sax_transform(row, w, a) for row in X])\n    X_fft = np.abs(fft(X, axis=1))\n\n    coeffs_cA, coeffs_cD = pywt.dwt(X, \'db1\', axis=1)\n    X_dwt = np.hstack((coeffs_cA, coeffs_cD))\n\n    X_paa = np.column_stack([X[:, i:i+2].mean(axis=1) for i in range(0, X.shape[1], 2)])\n\n    return {\n        "TS": X,\n        "FFT": X_fft,\n        "DWT": X_dwt,\n        "PAA": X_paa,\

### Function Transform_data Concatenate

In [4]:
# Transform data using TimeSeriesScalerMeanVariance and concatenate all transformed data
"""def transform_data(X, num_features=10):
    n_sax_symbols = int(X.shape[1] / num_features)
    n_paa_segments = int(X.shape[1] / num_features)

    X_fft = np.abs(fft(X, axis=1))

    coeffs_cA, coeffs_cD = pywt.dwt(X, 'db1', axis=1)
    X_dwt = np.hstack((coeffs_cA, coeffs_cD))

    paa = PiecewiseAggregateApproximation(n_segments=n_paa_segments)
    X_paa_ = paa.inverse_transform(paa.fit_transform(X))
    X_paa = X_paa_.reshape(X_paa_.shape[0], -1)

    sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
    X_sax_ = sax.inverse_transform(sax.fit_transform(X))
    X_sax = X_sax_.reshape(X_sax_.shape[0], -1)

    data = np.concatenate((X, X_fft, X_dwt, X_paa, X_sax), axis=1)
    data_concat = TimeSeriesScalerMeanVariance().fit_transform(data)
    data_concat.resize(data.shape[0], data.shape[1])

    return data_concat"""

"def transform_data(X, num_features=10):\n    n_sax_symbols = int(X.shape[1] / num_features)\n    n_paa_segments = int(X.shape[1] / num_features)\n\n    X_fft = np.abs(fft(X, axis=1))\n\n    coeffs_cA, coeffs_cD = pywt.dwt(X, 'db1', axis=1)\n    X_dwt = np.hstack((coeffs_cA, coeffs_cD))\n\n    paa = PiecewiseAggregateApproximation(n_segments=n_paa_segments)\n    X_paa_ = paa.inverse_transform(paa.fit_transform(X))\n    X_paa = X_paa_.reshape(X_paa_.shape[0], -1)\n\n    sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)\n    X_sax_ = sax.inverse_transform(sax.fit_transform(X))\n    X_sax = X_sax_.reshape(X_sax_.shape[0], -1)\n\n    data = np.concatenate((X, X_fft, X_dwt, X_paa, X_sax), axis=1)\n    data_concat = TimeSeriesScalerMeanVariance().fit_transform(data)\n    data_concat.resize(data.shape[0], data.shape[1])\n\n    return data_concat"

### Function Transform_data with normalization

In [5]:
# Transform data atualizado
def transform_data(X, num_features=10):
    n_sax_symbols = int(X.shape[1] / num_features)
    n_paa_segments = int(X.shape[1] / num_features)

    X_fft = np.abs(fft(X, axis=1))

    coeffs_cA, coeffs_cD = pywt.dwt(X, 'db1', axis=1)
    X_dwt = np.hstack((coeffs_cA, coeffs_cD))

    paa = PiecewiseAggregateApproximation(n_segments=n_paa_segments)
    X_paa_ = paa.inverse_transform(paa.fit_transform(X))
    X_paa = X_paa_.reshape(X_paa_.shape[0], -1)

    sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
    X_sax_ = sax.inverse_transform(sax.fit_transform(X))
    X_sax = X_sax_.reshape(X_sax_.shape[0], -1)

    data_X = TimeSeriesScalerMeanVariance().fit_transform(X)
    data_X.resize(data_X.shape[0], data_X.shape[1])
    
    data_FFT = TimeSeriesScalerMeanVariance().fit_transform(X_fft)
    data_FFT.resize(data_FFT.shape[0], data_FFT.shape[1])
    
    data_DWT = TimeSeriesScalerMeanVariance().fit_transform(X_dwt)
    data_DWT.resize(data_DWT.shape[0], data_DWT.shape[1])
    
    data_PAA = TimeSeriesScalerMeanVariance().fit_transform(X_paa)
    data_PAA.resize(data_PAA.shape[0], data_PAA.shape[1])
    
    data_SAX = TimeSeriesScalerMeanVariance().fit_transform(X_sax)
    data_SAX.resize(data_SAX.shape[0], data_SAX.shape[1])
    

    return {
        "TS": data_X,
        "FFT": data_FFT,
        "DWT": data_DWT,
        "PAA": data_PAA,
        "SAX": data_SAX
    }

In [6]:
def select_model(option, random_state):
    if option == '1nn':
        return KNeighborsTimeSeriesClassifier(distance='euclidean', n_neighbors=1)
    elif option == '3nn':
        return KNeighborsTimeSeriesClassifier(distance='euclidean', n_neighbors=3)
    elif option == 'svm':
        return SVC(C=100, gamma=0.01, kernel='linear', probability=True)
    elif option == 'rd':
        return RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
    else:
        return RandomForestClassifier(n_estimators=100,random_state=random_state)

### Train MetaModel Concatenate

In [7]:
"""def train_with_meta_classifier(X_train, y_train, base_option='svm', meta_option='random_forest', random_state=42):
    X_train_transformed = transform_data(X_train)

    loo = LeaveOneOut()
    loo.get_n_splits(X_train_transformed)

    # Treinar um modelo para todos os dados transformados
    model = select_model(base_option, random_state)
    scores = []
    for train_index, test_index in tqdm(loo.split(X_train_transformed), ascii=True, desc="Traning Instances"):
        X_train_fold, X_test_fold = X_train_transformed[train_index], X_train_transformed[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
        model.fit(X_train_fold, y_train_fold)
        score = model.score(X_test_fold, y_test_fold)
        scores.append(score)
    avg_score = np.mean(scores)

    # Preparar dados para o meta-classificador
    meta_features = []
    for X_trans in X_train_transformed:
        instance_features = []
        proba = model.predict_proba(X_trans.reshape(1, -1)) # Reshape para compatibilidade com predict_proba
        instance_features.extend(proba.flatten())
        meta_features.append(instance_features)

    meta_features = np.array(meta_features)

    # Treinar o meta-classificador
    meta_classifier = select_model(meta_option, random_state)
    meta_classifier.fit(meta_features, y_train)

    return model, meta_classifier
"""

'def train_with_meta_classifier(X_train, y_train, base_option=\'svm\', meta_option=\'random_forest\', random_state=42):\n    X_train_transformed = transform_data(X_train)\n\n    loo = LeaveOneOut()\n    loo.get_n_splits(X_train_transformed)\n\n    # Treinar um modelo para todos os dados transformados\n    model = select_model(base_option, random_state)\n    scores = []\n    for train_index, test_index in tqdm(loo.split(X_train_transformed), ascii=True, desc="Traning Instances"):\n        X_train_fold, X_test_fold = X_train_transformed[train_index], X_train_transformed[test_index]\n        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]\n        model.fit(X_train_fold, y_train_fold)\n        score = model.score(X_test_fold, y_test_fold)\n        scores.append(score)\n    avg_score = np.mean(scores)\n\n    # Preparar dados para o meta-classificador\n    meta_features = []\n    for X_trans in X_train_transformed:\n        instance_features = []\n        proba = model

In [8]:
"""from tqdm import tqdm  # Importar a biblioteca tqdm

def predict_with_meta_classifier(X_test, trained_base_model, trained_meta_classifier):
    predictions = []
    meta_features_test = []  # Inicialize uma lista para armazenar todos os meta-recursos dos dados de teste

    for i in tqdm(range(len(X_test)), ascii=True, desc="Testing Instances"):
        x_instance = X_test[i].reshape(1, -1)
        x_transformed = transform_data(x_instance)

        instance_features = []
        for X_trans in x_transformed:  # Iterar sobre as diferentes transformações
            proba = trained_base_model.predict_proba(X_trans.reshape(1, -1))
            instance_features.extend(proba.flatten())  # Estender a lista com todas as probabilidades

        meta_feature = np.array(instance_features).reshape(1, -1)
        predictions.append(trained_meta_classifier.predict(meta_feature)[0])  # Adicionar a previsão à lista de previsões

        meta_features_test.append(meta_feature.flatten())  # Adicionar meta-recursos da instância atual à lista

    # Converter a lista de meta-recursos dos dados de teste em um array numpy
    meta_features_test = np.array(meta_features_test)

    # Salvar todos os meta-recursos dos dados de teste em um arquivo CSV
    # np.savetxt("meta-features-test.csv", meta_features_test, delimiter=",")

    return predictions
"""

'from tqdm import tqdm  # Importar a biblioteca tqdm\n\ndef predict_with_meta_classifier(X_test, trained_base_model, trained_meta_classifier):\n    predictions = []\n    meta_features_test = []  # Inicialize uma lista para armazenar todos os meta-recursos dos dados de teste\n\n    for i in tqdm(range(len(X_test)), ascii=True, desc="Testing Instances"):\n        x_instance = X_test[i].reshape(1, -1)\n        x_transformed = transform_data(x_instance)\n\n        instance_features = []\n        for X_trans in x_transformed:  # Iterar sobre as diferentes transformações\n            proba = trained_base_model.predict_proba(X_trans.reshape(1, -1))\n            instance_features.extend(proba.flatten())  # Estender a lista com todas as probabilidades\n\n        meta_feature = np.array(instance_features).reshape(1, -1)\n        predictions.append(trained_meta_classifier.predict(meta_feature)[0])  # Adicionar a previsão à lista de previsões\n\n        meta_features_test.append(meta_feature.flatt

### Train MetaModel using Dict variance

In [9]:
def train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='1nn', random_state=42):
    trained_models = {}  # Salvar modelos treinados para cada transformação
    
    X_train_transformed = transform_data(X_train)  # Transformar todo o conjunto de treino

    # Treinar um modelo para cada transformação e salvar no dicionário
    for rep, X_trans in tqdm(X_train_transformed.items(), ascii=True, desc="Training Base Models"):
        model = select_model(base_option, random_state)
        loo = LeaveOneOut()
        for train_index, _ in loo.split(X_trans):
            model.fit(X_trans[train_index], y_train[train_index])
        trained_models[rep] = model
        
    # Preparar dados para o meta-classificador
    meta_features = []
    for i in range(X_train.shape[0]):
        instance_features = []
        for rep, model in trained_models.items():
            proba = model.predict_proba(X_train_transformed[rep][i].reshape(1, -1))
            instance_features.extend(proba.flatten())  # Estender a lista com todas as probabilidades
        meta_features.append(instance_features)
    
    meta_features = np.array(meta_features)
    #np.savetxt("meta-features-train.csv", meta_features, delimiter=",")
    
    # Treinar o meta-classificador
    meta_classifier = select_model(meta_option, random_state)
    meta_classifier.fit(meta_features, y_train)
    
    return trained_models, meta_classifier, meta_features


In [10]:
def predict_with_meta_classifier(X_test, trained_base_models, trained_meta_classifier):
    predictions = []
    meta_features_test = []  # Inicialize uma lista para armazenar todos os meta-recursos dos dados de teste
    
    for i in tqdm(range(len(X_test)), ascii=True, desc="Testing Instances"):
        x_instance = X_test[i].reshape(1, -1)
        x_transformed = transform_data(x_instance)
        
        instance_features = []
        for rep, model in trained_base_models.items():  
            proba = model.predict_proba(x_transformed[rep][0].reshape(1, -1))
            instance_features.extend(proba.flatten())  
        
        meta_feature = np.array(instance_features).reshape(1, -1)
        predictions.append(trained_meta_classifier.predict(meta_feature)[0])  
        
        meta_features_test.append(meta_feature.flatten())  
    
    meta_features_test = np.array(meta_features_test)

    return predictions, meta_features_test


In [12]:
train, train_labels = load_classification('Coffee', extract_path="./Temp/", split='TRAIN')
test, test_labels = load_classification('Coffee', extract_path="./Temp/", split='test')
xtrain = train.reshape(train.shape[0], -1)
xtest = test.reshape(test.shape[0], -1)
trained_base_models, meta_classifier, meta_feat = train_with_meta_classifier(xtrain, train_labels, base_option='svm', meta_option='rd', random_state=42)
predictions_test_meta, predict = predict_with_meta_classifier(xtest, trained_base_models, meta_classifier)

Testing Instances: 100%|##########| 28/28 [00:00<00:00, 186.42it/s]


### Train/Predict

In [ ]:
univariate_list = list(univariate_equal_length)
univariate_list.sort()

In [ ]:
dataset_quali_list = ['Adiac', 'Beef', 'Car', 'CBF', 'Coffee', 'DiatomSizeReduction', 'ECG200', 'ECGFiveDays', 'FaceFour','GunPoint', 'Lightning2', 'Lightning7', 'MedicalImages', 'MoteStrain', 'OliveOil', 'SonyAIBORobotSurface1','SonyAIBORobotSurface2', 'SyntheticControl', 'Trace', 'TwoPatterns']
dataset_full_list = ['Worms','FaceAll','SyntheticControl','SemgHandMovementCh2','Herring','GunPointAgeSpan','SmoothSubspace','SemgHandSubjectCh2','LargeKitchenAppliances','Plane','Fish','ScreenType','PhalangesOutlinesCorrect','CricketZ','MiddlePhalanxOutlineAgeGroup','ECG5000','Chinatown','ShapeletSim','MiddlePhalanxTW','Symbols','EOGHorizontalSignal','Ham','UMD','HouseTwenty','TwoPatterns','MiddlePhalanxOutlineCorrect','Wafer','Rock','DistalPhalanxTW','CricketY','SonyAIBORobotSurface1','FacesUCR','FiftyWords','Mallat','Strawberry','SwedishLeaf','ProximalPhalanxOutlineAgeGroup','DiatomSizeReduction','MixedShapesRegularTrain','Trace','ECGFiveDays','Lightning2','MoteStrain','SmallKitchenAppliances','GunPointOldVersusYoung','Wine','ECG200','ProximalPhalanxOutlineCorrect','WordSynonyms', 'RefrigerationDevices','Lightning7','Yoga','FaceFour','CinCECGTorso','Beef','OliveOil','ChlorineConcentration','ArrowHead','ToeSegmentation1','TwoLeadECG','ProximalPhalanxTW','InsectEPGSmallTrain','WormsTwoClass','PowerCons','Coffee','InsectEPGRegularTrain','GunPointMaleVersusFemale','DistalPhalanxOutlineCorrect','ItalyPowerDemand','InsectWingbeatSound','BME','NonInvasiveFetalECGThorax2','CricketX','Haptics','EOGVerticalSignal','MixedShapesSmallTrain','Meat','SemgHandGenderCh2','ToeSegmentation2','Adiac','Car','NonInvasiveFetalECGThorax1','FreezerSmallTrain','OSULeaf','GunPoint','Earthquakes','BirdChicken','HandOutlines','BeetleFly','SonyAIBORobotSurface2','CBF','ACSF1','DistalPhalanxOutlineAgeGroup','FreezerRegularTrain']
problematicos = ['Crop','EthanolLevel','ElectricDevices','FordB','ShapesAll','StarLightCurves','Phoneme', 'Computers','InlineSkate','PigAirwayPressure', 'PigCVP','FordA','MedicalImages','PigArtPressure', 'UWaveGestureLibraryX','UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'UWaveGestureLibraryAll']

In [ ]:
algos = ['1nn', '3nn','svm', 'rd', 'random_forest']
accuracy_data = []

for dataset_name in dataset_full_list:
    train, train_labels = load_classification(dataset_name, extract_path="./Temp/", split='TRAIN')
    test, test_labels = load_classification(dataset_name, extract_path="./Temp/", split='test')

    xtrain = train.reshape(train.shape[0], -1)
    xtest = test.reshape(test.shape[0], -1)
    
    # Treino
    
    for algo in algos:
        trained_base_models, meta_classifier = train_with_meta_classifier(xtrain, train_labels, base_option='svm', meta_option=rd, random_state=42)
    # Teste
        predictions_test_meta = predict_with_meta_classifier(xtest, trained_base_models, meta_classifier)
    # Resultado
        test_accuracy_meta = np.mean(predictions_test_meta == test_labels)
    accuracy_data.append({'clf':{algo}, 'Dataset Name': dataset_name, 'Accuracy': test_accuracy_meta})

accuracy_df = pd.DataFrame(accuracy_data)


Training Base Models:  80%|########  | 4/5 [19:48:31<4:41:07, 16867.11s/it] 

In [ ]:
accuracy_df.to_csv('model_acc_SVM+RF+TDV2.csv', index=False)